In [1]:
%run uvms_base.ipynb

jit after {'jit': True, 'jit_options': {'flags': '-Ofast'}, 'compiler': 'shell'}
number of joints = 4


In [2]:
# conditional function instances
# coupled forward dynamics of uvms
x_next_coupled, x_nextdd_coupled, states, u, dt, coordinates_ll, coordinates_ul, manipulator_p, vehicle_p, base_T, states_checks, u_checks  = rig_dyn.forward_dynamics(gravity=9.81, 
                                                                                    floating_base_id=tau_F,
                                                                                    floating_base_bias_f=b_F , 
                                                                                    J_uv = J_uv, 
                                                                                    coupled=True)
x_next_coupled_eval = cs.Function('UVMSnext_coupled', [states, u, dt, vehicle_p, manipulator_p, base_T , coordinates_ll, coordinates_ul], [x_next_coupled, u_checks, x_nextdd_coupled])


# decoupled forward dynamics of uvms
x_next_decoupled, x_nextdd_decoupled, states, u, dt, coordinates_ll, coordinates_ul, manipulator_p, vehicle_p, base_T, states_checks, u_checks  = rig_dyn.forward_dynamics(gravity=9.81, 
                                                                                    floating_base_id=tau_F,
                                                                                    floating_base_bias_f=b_F , 
                                                                                    J_uv = J_uv, 
                                                                                    coupled=False)

x_next_decoupled_eval = cs.Function('UVMSnext_decoupled', [states, u, dt, vehicle_p, manipulator_p, base_T ,coordinates_ll, coordinates_ul], [x_next_decoupled, u_checks, x_nextdd_decoupled])



UVMSnext_use_coupled = cs.Function.if_else('UVMSnext_use_coupled', x_next_coupled_eval, x_next_decoupled_eval)
UVMSnext_use_coupled.save('uvms_forward_dynamics.casadi')

floating_base found
floating_base found
floating_base found
floating_base found
floating_base found
floating_base found


In [3]:
# # c , cpp or matlab code generation for forward dynamics
# UVMSnext_use_coupled.generate("x_next_eval.c")
# os.system(f"gcc -fPIC -shared x_next_eval.c -o libUVMS_xnext.so")

0

In [4]:
st_x0 = cs.vertcat(0,0,0, 0,0,0, 1.91318, 1.33339, 0.99245, 3.16073,   0, 0, 0, 0, 0, 0,  0.00751799, -0.0073626, 0.00281749, 0.0222385)
u0 = cs.vertcat(0.0, 0, 0, 0, 0, 0, 0.0340454, -0.00441695, 0.0062739, -0.00505981)
coord_min = cs.vertcat(blue.pose_min, alpha.joint_min)
coord_max = cs.vertcat(blue.pose_max, alpha.joint_max)
UVMSnext_use_coupled(use_coupled, st_x0, u0, alpha.delta_t, blue.sim_p, alpha.sim_p, alpha.base_T0, coord_min, coord_max)

(DM([2.18825e-07, 2.51181e-07, 6.97983e-06, -1.42818e-05, 7.75939e-06, 3.03379e-06, 1.91365, 1.3333, 0.992578, 3.16057, 6.45433e-06, 7.36874e-06, -0.00109076, -0.000416911, 0.00022741, 8.85267e-05, 0.0136182, -0.00232471, 0.00369053, -0.00505981]),
 DM([0, 0, 0, 0, 0, 0, 0.0340454, -0.00441695, 0.0062739, -0.00505981]),
 DM([0.000204173, 0.000307202, 0.000197476, -0.0154003, 0.00677282, 0.0028588, 0.236833, 0.149249, 0.022863, -0.977171]))

In [5]:
st_x0 = cs.vertcat(0,0,0, 0,0,0, 1.91318, 1.33339, 0.99245, 3.16073,   0, 0, 0, 0, 0, 0,  0.00751799, -0.0073626, 0.00281749, 0.0222385)
u0 = cs.vertcat(0.0, 0, 0, 0, 0, 0, 0.0340454, -0.00441695, 0.0062739, -0.00505981)

UVMSnext_use_coupled(not use_coupled, st_x0, u0, alpha.delta_t, blue.sim_p, alpha.sim_p, alpha.base_T0, coord_min, coord_max)

(DM([0, 0, 0, 0, 0, 0, 1.91365, 1.3333, 0.992578, 3.16057, 0, 0, 0, 0, 0, 0, 0.0136182, -0.00232471, 0.00369053, -0.00505981]),
 DM([0, 0, 0, 0, 0, 0, 0.0340454, -0.00441695, 0.0062739, -0.00505981]),
 DM([0, 0, 0, 0, 0, 0, 0.236833, 0.149249, 0.022863, -0.977171]))